# Developing the Text Generation Model

First of all, the text generation that I will be using requires a prompt. For that I'll take the first 10 words of the 27k+ rows of data I have, and use those for the prompts. I'm okay with 10 characters being "predetermined". 

In [1]:
import random
random.seed(42)

import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from sklearn.model_selection import train_test_split

In [63]:
NAME_TOKEN = "<|name|>"
DEV_TOKEN = "<|developer|>"
PUB_TOKEN = "<|publisher|>"
DESC_TOKEN = "<|description|>"
GENRES_TOKEN = "<|genres|>"
GAME_TOKEN = "<|game|>"
END_TOKEN = "<|endoftext|>"

In [3]:
torch.cuda.is_available()

True

In [4]:
df = pd.read_csv("../data/datasetv2.csv").dropna()

In [5]:
df.head()

,appid,name,developer,publisher,genres,description,header_image
0,10,Counter-Strike,Valve,Valve,Action,Play the world's number 1 online action game. ...,https://steamcdn-a.akamaihd.net/steam/apps/10/...
1,20,Team Fortress Classic,Valve,Valve,Action,One of the most popular online action games of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...
2,30,Day of Defeat,Valve,Valve,Action,Enlist in an intense brand of Axis vs. Allied ...,https://steamcdn-a.akamaihd.net/steam/apps/30/...
3,40,Deathmatch Classic,Valve,Valve,Action,Enjoy fast-paced multiplayer gaming with Death...,https://steamcdn-a.akamaihd.net/steam/apps/40/...
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,Action,Return to the Black Mesa Research Facility as ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...


# Loading and Testing the Model

Hugging Face Transformers makes it really easy to load pretrained models.

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to("cuda")

In [ ]:
prompts = df.description.apply(lambda x: " ".join(x.split()[:10]))

In [ ]:
prompt = random.choice(prompts)
prompt_encoded = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    prompt_encoded,
    do_sample=True, 
    max_length=500, 
    top_k=50, 
    top_p=0.95,
    no_repeat_ngram_size=5,
)
output_decoded = tokenizer.decode(output[0])


print(prompt)
print(output_decoded)

In [ ]:
del tokenizer
del model

# Fine-Tuning the Model to Generate Video Game Titles (Names)

The output already looks fantastic, but let's fine-tune the model to get even better results.

I'm going to work on the video game name generation first as a POC. We can use a special token, for example `<|name|>` as a prompt instead of needing words to prompt the title generation.
 
See: https://towardsdatascience.com/natural-language-generation-part-2-gpt-2-and-huggingface-f3acb35bc86a

So all we really need to do is format our data in with the prompt token (for this task, `<|name|>`) and the end of text token, which is built into the pretrained tokenizer: `<|endoftext|>` and fine-tune the pretrained model.

In [ ]:
# Example formatted name
name = df.name[0]
formatted_name = f"{NAME_TOKEN}{name}{END_TOKEN}"
print(formatted_name)

In [12]:
def save_formatted(file, list_of_texts, start_token, end_token):
    for text in list_of_texts:
        formatted_text = f"{start_token}{text}{end_token}"
        file.write(formatted_text)

In [ ]:
# Split our data into train and validation
train, validation = train_test_split(df.name, train_size=0.85, random_state=42)

print("train count:", train.count())
print("validation count:", validation.count())

In [ ]:
# with open("../data/training/name_train.txt", "w") as f:
#     save_formatted(f, train, NAME_TOKEN, END_TOKEN)

In [ ]:
# with open("../data/training/name_val.txt", "w") as f:
#     save_formatted(f, validation, NAME_TOKEN, END_TOKEN)

# Testing the Fine-tuned Name Model

I used a GPU cloud provider to fine-tune the pretrained GPT2-medium model using the `scripts/train-name.sh` script.

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained("../data/models/gpt2-name")
model = GPT2LMHeadModel.from_pretrained("../data/models/gpt2-name", pad_token_id=tokenizer.eos_token_id).to("cuda")

In [ ]:
prompt_encoded = tokenizer.encode(NAME_TOKEN, return_tensors="pt").to("cuda")

output = model.generate(
    prompt_encoded,
    do_sample=True, 
    max_length=500, 
    top_k=50, 
    top_p=0.95,
    no_repeat_ngram_size=5,
)
output_decoded = tokenizer.decode(output[0])


print(output_decoded)

This looks great, now I'll train a model that can generate each type of text we need.

In [ ]:
del tokenizer
del model

# Compiling the Dataset for All Types of Generation

We will have one unified dataset that has text that has the tokens: `<|name|>`, `<|developer|>`, `<|publisher|>`, `<|description|>`, `<|genres|>`, and of course `<|endoftext|>`.

Each example will consist of one of the class tokens, then some text, and then the end token. It would be nice to be able to generate all the text for a game with a single `<|game|>` token, so I'll also test training a model to do that.

In [13]:
columns = {
    NAME_TOKEN: df.name,
    DEV_TOKEN: df.developer,
    PUB_TOKEN: df.publisher,
    DESC_TOKEN: df.description,
    GENRES_TOKEN: df.genres
}

## Separate Token Dataset

Here I am getting only the unique values because I don't need the outputs to be representative of the distribution dataset itself, but better representative of the data itself.

In [42]:
corpus = []
labels = []

for start_token, col in columns.items():
    values = col.unique().tolist()
    corpus.extend(
        [f"{start_token}{value}{END_TOKEN}" for value in values]
    )
    labels.extend(
        [start_token for _ in values]
    )

In [43]:
len(corpus), len(labels)

(87027, 87027)

We can validate the length of the corpus makes sense because we have ~27k examples, and 5 columns with many of the values in 3 of the columns being duplicates.

In [41]:
expected_length_corpus = 0
for _, v in columns.items():
    print(v.name, v.count(), len(v.unique()))
    expected_length_corpus += len(v.unique())

print("expected_length_corpus:", expected_length_corpus)

name 27048 27006
developer 27048 17095
publisher 27048 14344
description 27048 27030
genres 27048 1552
expected_length_corpus: 87027


Here we split the corpus into the train and validation sets. Notice that we stratify using the labels to get a proportional number of each class in each set.

In [48]:
train, val = train_test_split(corpus, train_size=0.85, shuffle=True, stratify=labels, random_state=42)

In [60]:
len(train), train[:5]

(73972,
 ['<|developer|>Seattletek<|endoftext|>',
  '<|name|>Pen Island VR<|endoftext|>',
  "<|description|>What is Deadly Escape?Deadly Escape is a small episodic Survival Horror game where you have to beat each chapter with one life, inspired by survival horror games of the '90s.After the complex was raided by undead monsters, our wounded protagonist is abandoned inside the infirmary. He wakes up only to find that everyone has been massacred and the place is locked down. Now he has to trace the steps of the deceased that tried to escape before him, hoping to find a way out of this nightmare.FEATURES Old school survival horror action. Ration your ammo, every bullet counts ! Collect and use key items to advance deeper inside the complex. Explore and find new weapons to increase your odds of survival. Find documents, files, and notes from the deceased personal. Uncover what happened! Compete against other players via online leaderboards! Who will be the first survivor? Listen to an all 

In [59]:
len(val), val[:5]

(13055,
 ['<|description|>Heaven Island is an experiential game in which you\'ll explore wonderful environments and places.On this Island, time and space are frozen and you\'ll be able to visit it and at the same time explore the inner of your soul.All problems and worries of life will be left far behind so that you\'ll be able to rest your soul and mind during this experience.The inspiration for the environments comes from the Renzo Piano designs and from the book "Origins of Architectural Pleasure" in which the player\'s welfare is of primary importance.This game is a Massive Multiplayer Online experience in the sense that you\'ll be able to connect with all other gamers and will be able to view their essence in order to be able to create a "Connection". All the details you see on the Island are made by hand, every rock you see is put there with a purpose.We really hope you\'ll enjoy this wonderful experience.Designed to be played with or without the Oculus VR, Gear VR and (soon) the

Here is a horribly inefficient and hacky way of validating the stratification.

In [55]:
pd.Series(train).apply(lambda x: x.split("<|")[1].split("|>")[0]).value_counts(normalize=True)

description    0.310590
name           0.310320
developer      0.196439
publisher      0.164819
genres         0.017831
dtype: float64

In [56]:
pd.Series(val).apply(lambda x: x.split("<|")[1].split("|>")[0]).value_counts(normalize=True)

description    0.310609
name           0.310303
developer      0.196400
publisher      0.164841
genres         0.017848
dtype: float64

Close enough :) Now we just save our train and val sets to two text files again and train a new model with this data.

In [61]:
# with open("../data/training/all_train.txt", "w") as f:
#     f.write("".join(train))

In [62]:
# with open("../data/training/all_val.txt", "w") as f:
#     f.write("".join(val))

## Dataset for One-Shot Generation

Now I'll create another corpus that wraps all of the fields and should allow us to generate games with more cohesive attributes. I also don't expect there to be too many duplicate games in the dataset so I haven't removed any for this dataset.

I'm less optimistic about this approach, but we'll see what happens.

In [71]:
corpus = []

for idx, row in df.iterrows():
    game_text = (f"{GAME_TOKEN}{NAME_TOKEN}{row['name']}{DEV_TOKEN}{row.developer}"
                 f"{PUB_TOKEN}{row.publisher}{DESC_TOKEN}{row.description}{END_TOKEN}")
    
    corpus.append(game_text)

In [75]:
corpus[100:105]

["<|game|><|name|>Bejeweled Twist<|developer|>PopCap Games, Inc.<|publisher|>PopCap Games, Inc.<|description|>Spin, match, explode... WOW! It's a brilliant new way to play Bejeweled! Get set for a vivid sensory rush as you spin and match explosive gems for shockwaves of fun. Rotate jewels freely to set up electrifying combos, outwit surprising obstacles like Locks and Bombs, and create high-voltage Flame and Lightning gems. When you need to dial up the intensity or fine-tune your skills, turn to Challenge mode or five-minute Blitz. And if relaxing is more your style, kick up your feet with stress-free Zen. No matter the mode, you'll discover new strategies, improve your moves, and find endless ways to win! Use the revolutionary Gem Rotator to move gems around and make matches anywhere on the board Relax, rev up, refocus or recharge in four game modes — Classic, Zen, Challenge and Blitz Clear away Locks and Bombs for magnificent bonuses Create Flame and Lightning power gems, then blast 

In [76]:
train, val = train_test_split(corpus, train_size=0.85, shuffle=True, random_state=42)

In [79]:
len(train), len(val)

(22990, 4058)

In [77]:
# with open("../data/training/game_train.txt", "w") as f:
#     f.write("".join(train))

In [78]:
# with open("../data/training/game_val.txt", "w") as f:
#     f.write("".join(val))